# Extreact reactions from mol entries

In [1]:
import numpy as np
from pymatgen.reaction_network.reaction import bucket_mol_entries, unbucket_mol_entries
from pymatgen.reaction_network.reaction import IntramolSingleBondChangeReaction, IntermolecularReaction
from monty.serialization import dumpfn, loadfn

### Load molecule entries 

In [2]:
num_entries = 2000
#num_entries = None
fname = f'/Users/mjwen/Documents/Dataset/rxnrep/mol_entries_n{num_entries}.json'

mol_entries = loadfn(fname)

print('number of mol entries loaded:', len(mol_entries))

number of mol entries loaded: 862


### Intra-molecular bond breaking reactions, i.e. A->B

In [3]:
buckets = bucket_mol_entries(mol_entries)
intra_mol_reactions, _ = IntramolSingleBondChangeReaction.generate(buckets)
len(intra_mol_reactions)

/Users/mjwen/Applications/pymatgen/pymatgen/reaction_network/reaction.py:2759: FutureWarning: Nbonds is deprecated
`Nbonds` is replaced by `num_bonds`. This will be removed shortly.
  v = getattr(m, k)
/Users/mjwen/Applications/pymatgen/pymatgen/reaction_network/reaction.py:682: FutureWarning: edges is deprecated
`edges` is replaced by `bonds`. This will be removed shortly.
  for edge in entry1.edges:


71

### Inter-molecular bond breaking reactions, i.e. A->B+C

In [4]:
buckets = bucket_mol_entries(mol_entries)
inter_mol_reactions, _ = IntermolecularReaction.generate(buckets)
len(inter_mol_reactions)

/Users/mjwen/Applications/pymatgen/pymatgen/reaction_network/reaction.py:968: FutureWarning: edges is deprecated
`edges` is replaced by `bonds`. This will be removed shortly.
  for edge in entry.edges:


36

### write reactions to file (pymatgen reactions)

In [5]:
def split_train_val_test(data,
                         val_ratio=0.1,
                         test_ratio=0.1,
                         random_seed=35):
    """
    Split dataset into training, validation, and test test.
    """
    assert val_ratio + test_ratio < 1.0, "validation + test >= 1"
    size = len(data)
    num_val = int(size * val_ratio)
    num_test = int(size * test_ratio)
    num_train = size - num_val - num_test

    if random_seed is not None:
        np.random.seed(random_seed)

    idx = np.random.permutation(size)
    train_idx = idx[:num_train]
    val_idx = idx[num_train:num_train + num_val]
    test_idx = idx[num_train + num_val:]

    train_set = [data[i] for i in train_idx]
    val_set = [data[i] for i in val_idx]
    test_set = [data[i] for i in test_idx]

    return train_set, val_set, test_set


reactions = intra_mol_reactions + inter_mol_reactions
train_set, val_set, test_set = split_train_val_test(reactions)


fname = f'/Users/mjwen/Documents/Dataset/rxnrep/reactions_n{num_entries}_train.json'
dumpfn(train_set, fname)

fname = f'/Users/mjwen/Documents/Dataset/rxnrep/reactions_n{num_entries}_val.json'
dumpfn(val_set, fname)

fname = f'/Users/mjwen/Documents/Dataset/rxnrep/reactions_n{num_entries}_test.json'
dumpfn(test_set, fname)